In [1]:
import talib as ta
from datetime import datetime
from datetime import timedelta
from bigtrader.constant import Direction, OrderType

STRATEGY_NAME = "STRATEGY_1m"

# def on_timer(context, data=None):
#     """定时器事件(每秒回调一次)"""
#     context.counter += 1
#     if context.counter == 15:
#         rv = context.order("IF2106.CFX", 1, 5054, order_type="MARKET")
#         print(">>>>>>>>>>>>start order1", rv)
#         rv = context.order("P2109.DCE", 2, 8000, order_type="MARKET")
#         print(">>>>>>>>>>>>start order2", rv)
#         print(context.get_pending_order_reqs(context.get_custom_id()))
        
def initialize(context):
    """初始化"""
    print("initialize")  
    context.symbol = instruments
    context.my_instruments = context.get_conf_param("instruments")#从传入参数中获取需要交易的合约
    context.closetime_day = context.get_conf_param("closetime_day")#日内策略白盘平仓时间，一般14:58
    context.closetime_night = context.get_conf_param("closetime_night")#日内策略夜盘平仓时间，一般22:58，注意有些商品夜盘收盘时间不一样
    context.order_num = context.get_conf_param("order_num")#下单手数
    context.set_universe(context.my_instruments)#设置需要处理的合约

#     # 如果是非交易时间段，等到上午9点或晚上21点再执行algo()
#     schedule(schedule_func = algo, date_rule = '1d', time_rule = '09:00:00')
#     schedule(schedule_func = algo, date_rule = '1d', time_rule = '21:00:00')
        
def before_trading(context, data):
    """盘前处理"""
    #print("before_trading")
    context.subscribe(context.my_instruments) #注册合约
    context.index = 0
    context.current_instruments = []
    context.flag = 0 #用于获取今开
    context.count = 0
#     print(context.current_instruments)
#     context.history_data = data.history(context.current_instruments[0], ["high","low","open"], 2, "1d")

    for instr in context.my_instruments:
        index = instr.find('.')
        #获取当前主力合约，如果含有.即指定某个具体合约，否则认为是需要处理主连合约
        if index<=0:
            context.current_instruments.append(data.current_dominant(instr))
        else:#使用指定合约
            context.current_instruments.append(instr)                                   
    print("当前主力合约", context.current_instruments)
    context.flag = context.flag + 1
    
    
def handle_data(context, data):
    """Bar行情推送"""
    #print("handle_data:", data.current_dt)
    #获取当前时间
    if context.flag == 1 :
        context.history_data = data.history(context.current_instruments[0], ["high","low","open"], 1, "1m")
        context.flag = context.flag + 1
        context.today_open = context.history_data.iloc[-1]['open'] 
        
    cur_date =  data.current_dt
    cur_hm = cur_date.strftime('%H:%M')
    time = cur_hm

    #部分品种夜盘收盘时间不一样，此时间表示指定的尾盘平仓时间往后偏移30分钟，这段时间内不能开新仓，只能平仓。给30分钟是为了足够的冗余
    closetime_nightshift = (datetime.strptime(context.closetime_night,'%H:%M') + timedelta(minutes = 30)).strftime('%H:%M')
    
    # 开盘价直接在data最后一个数据里取到,前一交易日的最高和最低价为history_data里面的倒数第二条中取到
    context.history_data_2 = data.history(context.current_instruments[0], ["high","low","open"], 1, "1d")
    context.high = context.history_data_2.iloc[0]['high']
    context.low = context.history_data_2.iloc[0]['low']

    for instr in context.current_instruments:
        # 分别获取多头持仓，和空头持仓
        position_long = context.get_position(instr, Direction.LONG)
        position_short = context.get_position(instr, Direction.SHORT)
        # 获取当前价格
        price = data.current(instr, "close")
        
        #尾盘平仓
        if((cur_hm>=context.closetime_day and cur_hm<="15:00") or (cur_hm>=context.closetime_night and cur_hm<=closetime_nightshift)):
            if(position_long.current_qty != 0):
                rv = context.sell_close(instr, position_long.avail_qty, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 尾盘平多 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) #输出关键日志
            if(position_short.current_qty != 0):
                rv = context.buy_close(instr, position_short.avail_qty, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 尾盘平空 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) #输出关键日志
            #尾盘不开新仓，直接返回
            continue

        # 多头持仓小于开盘价止损 平多
        c1 = position_long.avail_qty != 0 and price < context.today_open
        # 空头持仓大于开盘价止损 平空
        c2 = position_short.avail_qty != 0 and price > context.today_open
        # 当前的最新价大于了前一天的最高价且没有持仓 开多
        c3 = (position_short.avail_qty == 0 and position_long.avail_qty == 0) and (price > context.high and not context.count)
        # 当前最新价小于了前一天的最低价且没有持仓 开空
        c4 = (position_short.avail_qty == 0 and position_long.avail_qty == 0) and (price < context.low and not context.count)

        if c1:
            rv = context.sell_close(instr, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 平多 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
            
        if c2:
            rv = context.buy_close(instr, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 平空 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志

        if c3:
            rv = context.buy_open(instr, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开多 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
            context.count = 1 #只开仓一次的判断
            context.write_log(msg, stdout=1) #输出关键日志

        if c4:
            rv = context.sell_open(instr, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开空 for " + instr + " 最新价=" + str(price) + " rv:" + str(rv)
            context.count = 1
            context.write_log(msg, stdout=1) #输出关键日志
                
#             # 交易逻辑部分
#             if position_long: 。
#                 if price < open:
#                     order_volume(symbol=context.symbol, volume=1, side=OrderSide_Sell,
#                                  order_type=OrderType_Market, position_effect=PositionEffect_Close)
#                     print('以市价单平多仓')
#             elif position_short: # 空头持仓大于开盘价止损。
#                 if bar.close > open:
#                     order_volume(symbol=context.symbol, volume=1, side=OrderSide_Buy,
#                                  order_type=OrderType_Market, position_effect=PositionEffect_Close)
#                     print('以市价单平空仓')
#             else:  # 没有持仓。
#                 if bar.close > high and not context.count:  # 当前的最新价大于了前一天的最高价
#                     # 开多
#                     order_volume(symbol=context.symbol, volume=1, side=OrderSide_Buy,
#                                  order_type=OrderType_Market, position_effect=PositionEffect_Open)
#                     print('以市价单开多仓')
#                     context.count = 1
#                 elif bar.close < low and not context.count:  # 当前最新价小于了前一天的最低价
#                     # 开空
#                     order_volume(symbol=context.symbol, volume=1, side=OrderSide_Sell,
#                                  order_type=OrderType_Market, position_effect=PositionEffect_Open)
#                     print('以市价单开空仓')
#                     context.count = 1

            
def handle_order(context, order):
    """委托回报推送"""
    msg = "handle_order:" + order.log_str()
    context.write_log(msg, stdout=1) #输出关键日志
    
        
def handle_trade(context, trade):
    """成交回报推送"""
    msg = "handle_trade:" + trade.log_str()
    context.write_log(msg, stdout=1) #输出关键日志
    
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(trade.instrument, Direction.LONG)
    position_short = context.get_position(trade.instrument, Direction.SHORT)
    msg = "当前多头持仓:"+ str(position_long) + "当前空头持仓:"+ str(position_short)
    context.write_log(msg, stdout=1) #输出关键日志
    


instruments = ['RB2110.SHF']#["RB2110.SHF"] #["RB2110.SHF"] [I2105.DCE"] ['RB']# RB2101.SHF 如果想用主连合约，可以出入合约简码，例如RB,I,CF
#需要交易者传入的参数
strategy_setting = [
    {
        "instruments": instruments,
        "order_num": 2,
        "closetime_day": "14:58",
        "closetime_night": "22:58"
    }    
    
]
start_date = "2021-04-21"
end_date = "2021-04-27"
md = M.hfbacktest.v1(start_date=start_date,
                     end_date=end_date,
                     instruments=instruments,
                     capital_base=100000,
                     product_type=Product.FUTURE,
                     frequency=Frequency.MINUTE,
                     initialize=initialize,
                     before_trading_start=before_trading,
                     handle_data=handle_data,
                     handle_order=handle_order,
                     handle_trade=handle_trade,
                     plot_charts=True,
                     volume_limit=1.0,
                     disable_cache=0,
                     show_debug_info=1,
                     strategy_setting=strategy_setting,
                     slippage_type=SlippageType.FIXED,#滑点固定模式
                     slippage_value=1.0,#买卖双向各1个滑点
                     m_deps=np.random.rand())

[2021-05-20 13:40:45.823948] INFO: moduleinvoker: hfbacktest.v1 开始运行..

[2021-05-20 13:40:45.849337] INFO: hfbacktest: passed-in daily_data_ds:None

[2021-05-20 13:40:45.853954] INFO: hfbacktest: passed-in minute_data_ds:None

[2021-05-20 13:40:45.856170] INFO: hfbacktest: passed-in tick_data_ds:None

[2021-05-20 13:40:45.858168] INFO: hfbacktest: passed-in each_data_ds:None

...

当前主力合约 ['RB2110.SHF']
2021-05-20 13:40:48.016577 strategy_20210520(bkt999,): handle_order:[bkt999,5,RB2110.SHF,long,open,0,2,5228.0,pending,21:01:00,5] 
2021-05-20 13:40:48.022139 strategy_20210520(bkt999,): 2021-04-22 21:01:00 开多 for RB2110.SHF 最新价=5228.0 rv:0 
2021-05-20 13:40:48.026402 strategy_20210520(bkt999,): handle_order:[bkt999,5,RB2110.SHF,long,open,2,2,5228.0,filled,21:01:00,5] 
2021-05-20 13:40:48.027336 strategy_20210520(bkt999,): handle_trade:[bkt999,5,RB2110.SHF,long,open,2,5230.0,SHFE.5,21:02:00.000000] 
2021-05-20 13:40:48.027513 strategy_20210520(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-20 13:40:48.031226 strategy_20210520(bkt999,): handle_order:[bkt999,6,RB2110.SHF,short,close_today,0,2,5222.0,pending,21:02:00,6] 
2021-05-20 13:40:48.032187 strategy_20210520(bkt999,): 2021-04-22 21:02:00 平多 for RB211

当前主力合约 ['RB2110.SHF']
2021-05-20 13:40:48.831401 strategy_20210520(bkt999,): handle_order:[bkt999,7,RB2110.SHF,long,open,0,2,5297.0,pending,21:01:00,7] 
2021-05-20 13:40:48.831701 strategy_20210520(bkt999,): 2021-04-23 21:01:00 开多 for RB2110.SHF 最新价=5297.0 rv:0 
2021-05-20 13:40:48.835080 strategy_20210520(bkt999,): handle_order:[bkt999,7,RB2110.SHF,long,open,2,2,5297.0,filled,21:01:00,7] 
2021-05-20 13:40:48.835768 strategy_20210520(bkt999,): handle_trade:[bkt999,7,RB2110.SHF,long,open,2,5298.0,SHFE.7,21:02:00.000000] 
2021-05-20 13:40:48.836015 strategy_20210520(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-20 13:40:48.840344 strategy_20210520(bkt999,): handle_order:[bkt999,8,RB2110.SHF,short,close_today,0,2,5297.0,pending,21:02:00,8] 
2021-05-20 13:40:48.840852 strategy_20210520(bkt999,): 2021-04-23 21:02:00 平多 for RB211

2021-05-20 13:40:50.103488 strategy_20210520(bkt999,): handle_order:[bkt999,10,RB2110.SHF,short,close_today,0,2,5402.0,pending,22:58:00,10] 
2021-05-20 13:40:50.103844 strategy_20210520(bkt999,): 2021-04-26 22:58:00 尾盘平多 for RB2110.SHF 最新价=5402.0 rv:0 
2021-05-20 13:40:50.123330 strategy_20210520(bkt999,): handle_order:[bkt999,10,RB2110.SHF,short,close_today,2,2,5402.0,filled,22:58:00,10] 
2021-05-20 13:40:50.124203 strategy_20210520(bkt999,): handle_trade:[bkt999,10,RB2110.SHF,short,close_today,2,5402.0,SHFE.10,22:59:00.000000] 
2021-05-20 13:40:50.124400 strategy_20210520(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-20 13:40:50.712131 run_backtest() finished! time cost 4.226s! 


[2021-05-20 13:40:51.793842] INFO: hfbacktest: backtest done, raw_perf_ds:DataSource(3e3684489c83483982e8e143b0de947fT)

[2021-05-20 13:40:53.164030] INFO: moduleinvoker: hfbacktest.v1 运行完成[7.340155s].